<a href="https://colab.research.google.com/github/stebechoi/CP2/blob/Hwkdir/3.%20(MovieLense100K)_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MovieLense 데이터를 이용한 영화 추천 시스템(Collaborative filtering: 협업필터링)

## CF
- Collaborative Filtering 협업 필터링 </br>
  사용자와 비슷한 다른 사용자의 평가를 파악하여 이용
    - 장점: 아이템 특성에 의존하지 않는다.
    - 단점
      - 평가되지 않은 아이템은 추천하지 않는다(new item problem)
      - 보통 가장 인기 있는 아이템이 추천된다.
      - 비슷한 유저 군이 존재하는 사용자 그룹이 필요하다.(cold start problem for new users)

### 데이터 로드

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/gdrive')
# 작업 디렉토리 변경 change directory
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/CP/CP2')

Mounted at /content/gdrive


In [5]:
df = pd.read_csv('movielense_rt100.csv')
df

,user_id,movie_id,movie_title,rating,TotalRatingCount,release_year,release_month,rating_year,rating_month,age_group,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,308,1,Toy Story (1995),4,452,1995,1,1998,2,6,...,0,0,0,0,0,0,0,0,0,0
1,308,4,Get Shorty (1995),5,209,1995,1,1998,2,6,...,0,0,0,0,0,0,0,0,0,0
2,308,7,Twelve Monkeys (1995),4,392,1995,1,1998,2,6,...,0,0,0,0,0,0,1,0,0,0
3,308,8,Babe (1995),5,219,1995,1,1998,2,6,...,0,0,0,0,0,0,0,0,0,0
4,308,9,Dead Man Walking (1995),4,299,1995,1,1998,2,6,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65003,729,346,Jackie Brown (1997),1,126,1997,1,1998,4,1,...,0,0,0,0,0,0,0,0,0,0
65004,729,690,Seven Years in Tibet (1997),2,155,1997,1,1998,4,1,...,0,0,0,0,0,0,0,0,1,0
65005,729,748,"Saint, The (1997)",4,316,1997,3,1998,4,1,...,0,0,0,0,0,1,0,1,0,0
65006,729,751,Tomorrow Never Dies (1997),3,180,1997,1,1998,4,1,...,0,0,0,0,0,1,0,1,0,0


In [11]:
data = df[['user_id', 'movie_id', 'rating']].sort_values(['movie_id'], ascending=True)
data

,user_id,movie_id,rating
0,308,1,4
6085,141,1,3
35008,542,1,4
34972,779,1,4
34864,882,1,5
...,...,...,...
13319,291,1047,2
881,181,1047,2
28740,872,1047,4
11605,472,1047,4


In [14]:
index_movie = df[['movie_id', 'movie_title']].sort_values(['movie_id']).drop_duplicates(subset=['movie_id']).reset_index(drop=True)
index_movie

,movie_id,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,4,Get Shorty (1995)
3,7,Twelve Monkeys (1995)
4,8,Babe (1995)
...,...,...
337,928,"Craft, The (1996)"
338,1012,Private Parts (1997)
339,1016,Con Air (1997)
340,1028,Grumpier Old Men (1995)


### CF 유사도 모델


#### 참고
- https://www.youtube.com/watch?v=mJ8pEjN3FKg </br>
- https://www.youtube.com/watch?v=Lc5mfCF0mCU

In [ ]:
# 협업필터링을 이용한 영화 추천 엔진
# matrix 구성. 이 방법은 각 id가 순차적으로 있는 경우 사용 가능.
# 앞서 rating갯수가 100개 이하인 데이터 삭제로 인해 movie_id가 일부 건너뛰고 존재하여 다음과 같은 에러 발생
# (영화 갯수가 342개이나, movie_id는 1047까지 있다)
# index 346 is out of bounds for axis 1 with size 342

# n_users = data['user_id'].unique().shape[0] #943
# n_items = data['movie_id'].nunique() #342 (rating 100개 이상인 영화)

# #(943,342)형태의 0으로 이루어진 행렬
# data_matrix = np.zeros((n_users, n_items)) 

# # sparse matrix 형태로 구성, 위 zeros행렬에 각 값 대치
# for row in data.itertuples():
#   data_matrix[row[1]-1, row[2]-1]=row[3]
# # itertuple이용 행에 접근할 수 있는 iterator 생성
# # -1은 인덱스와 다르기 때문
# # row[1]-1: 사용자정보, row[2]-1: 영화정보, row[3]:평점

# data_matrix

# cosine similarity가 아닌 cosine distances 활용한 코드 참고
# # 사용자 간 유사도 행렬 생성
# from sklearn.metrics.pairwise import cosine_distances

# cosine_distances(train)

# # 코사인 유사도: 내적 공간의 두 벡터의 사이각에 대한 코사인을 측정해 유사도를 나타내는 척도
# # cos(0) == 1, cos(θ) < 1
# # cosine distance함수로 벡터만 주면 바로 계산 가능
# distances = 1 - cosine_distances(train) # 자기자신을 1로 하도록 변경

# # 평가 예측 및 모델 성능 측정
# user_pred = distances.dot(train) / np.array([np.abs(distances).sum(axis=1)]).T
# # np.dot(). 행의 크기가 같아야 연산 가능
# # 

# from sklearn.metrics import mean_squared_error
# def get_mse(pred, actual):
#   pred = pred[actual.nonzero()].flatten()
#   actual = actual[actual.nonzero()].flatten()
#   return mean_squared_error(pred, actual)

# # 훈련데이터 RMSE
# a = np.sqrt(get_mse(user_pred, train)) 
# # 테스트 RMSE
# b = np.sqrt(get_mse(user_pred, test)) 

# print(a,b)

#### User-based

In [16]:
data_matrix = data.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0)

data_matrix

movie_id,1,2,4,7,8,9,11,12,13,14,...,865,866,879,895,926,928,1012,1016,1028,1047
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,3.0,4.0,1.0,5.0,2.0,5.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
940,0.0,0.0,2.0,4.0,5.0,3.0,0.0,4.0,0.0,3.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# 훈련 데이터와 테스트 데이터의 분리
from sklearn.model_selection import train_test_split

# 왜 train, test = train_test_split(data_matrix, test_size=0.2, random_state=42) 와 같이 data_matrix에서 나누는 게 아니라 다음과 같이 사용하는가?
train, val = train_test_split(data, test_size=0.2, random_state=42)

# train 데이터 만으로 다시 추천 데이터 생성
user_item_matrix = train.pivot('user_id', 'movie_id', 'rating').fillna(0)
user_item_matrix

<ipython-input-28-333de561f826>:8: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  user_item_matrix = train.pivot('user_id', 'movie_id', 'rating').fillna(0)


movie_id,1,2,4,7,8,9,11,12,13,14,...,865,866,879,895,926,928,1012,1016,1028,1047
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,3.0,4.0,0.0,5.0,2.0,5.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
940,0.0,0.0,2.0,4.0,5.0,0.0,0.0,4.0,0.0,3.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# 사용자 간 유사도 행렬 생성
from sklearn.metrics.pairwise import cosine_similarity

# 유저 유사도 shape : 유저수 * 유저수
user_sim = cosine_similarity(user_item_matrix, user_item_matrix)
user_sim = pd.DataFrame(data= user_sim, columns= user_item_matrix.index, index= user_item_matrix.index)
user_sim

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.177306,0.083164,0.072254,0.322891,0.480764,0.456933,0.252580,0.023898,0.380789,...,0.440399,0.096905,0.283524,0.161344,0.165939,0.128963,0.346082,0.203582,0.164315,0.388107
2,0.177306,1.000000,0.044029,0.138416,0.117779,0.219716,0.114874,0.069503,0.133193,0.214820,...,0.142556,0.394106,0.425973,0.459951,0.276577,0.260420,0.215980,0.182962,0.158388,0.088825
3,0.083164,0.044029,1.000000,0.462958,0.045793,0.090442,0.079678,0.062829,0.137603,0.091002,...,0.053456,0.078084,0.199410,0.100233,0.062166,0.052314,0.121586,0.181932,0.112252,0.054175
4,0.072254,0.138416,0.462958,1.000000,0.017275,0.066900,0.124833,0.150832,0.082397,0.069478,...,0.051215,0.054550,0.159485,0.140048,0.115562,0.048729,0.135253,0.207980,0.094104,0.045416
5,0.322891,0.117779,0.045793,0.017275,1.000000,0.263613,0.390138,0.286522,0.090468,0.164532,...,0.377806,0.103316,0.136076,0.129845,0.108221,0.065540,0.280858,0.216933,0.151819,0.328572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.128963,0.260420,0.052314,0.048729,0.065540,0.122242,0.109878,0.041708,0.000000,0.066451,...,0.052417,0.459948,0.330723,0.262406,0.506843,1.000000,0.088784,0.159419,0.048177,0.158237
940,0.346082,0.215980,0.121586,0.135253,0.280858,0.340045,0.371399,0.214751,0.175533,0.335277,...,0.266483,0.079512,0.198208,0.116896,0.177842,0.088784,1.000000,0.130199,0.206717,0.241879
941,0.203582,0.182962,0.181932,0.207980,0.216933,0.190125,0.060792,0.128169,0.130698,0.136406,...,0.057120,0.267695,0.268595,0.219058,0.137028,0.159419,0.130199,1.000000,0.068541,0.096481


In [36]:
result = []

# train data의 전체 평균 평점 => 체킹이 안된 경우 이 값으로 대체
global_average = train['rating'].mean()

# val set에 있는 영화의 평점 예측하여 rmse 계산
from tqdm.notebook import tqdm # 반복문 진행율 표시 프로그레스 바

for _, row in tqdm(val.iterrows()):
  user = row['user_id']
  movie = row['movie_id']
  rating = row['rating']

  if movie in user_item_matrix.columns and user in user_item_matrix.index:

    # 영화를 본 다른 유저
    movie_ratings = user_item_matrix[user_item_matrix[movie]> 0][movie]
    # 유저목록
    user_ids = movie_ratings.index

    # 예측하려는 유저와 해당 영화에 평점을 매긴 다른 유저들간의 유사도 # 기준 설정을 따로 안했는데, 어떤 것을 기준?
    similarity = user_sim[user_ids].loc[user]

    # 위에서 구한 유사도를 가중치로 사용하여 다른 유저들의 평점의 평균을 구한다.
    sim_rate = np.dot(movie_ratings.values, similarity) # value로 치면 series는 가질 수 없다고 나온다. values임에 유의
    sim = similarity.sum() # 무슨 의미인가, 아래 나누기는 왜 하는지

    if sim == 0 :
      continue
      predicted_rating = sim_rate / sim
      result.append([user, movie, rating, predicted_rating])
    else:
      result.append([user, movie, rating, global_average])

weighted_average_result_df = pd.DataFrame(result, columns = ['user_id', 'movie_id', 'rating', 'predictedRating'])

0it [00:00, ?it/s]

In [34]:
movie_ratings

user_id
5      2.0
13     2.0
15     3.0
38     5.0
43     2.0
      ... 
896    1.0
907    5.0
932    2.0
934    2.0
937    2.0
Name: 225, Length: 85, dtype: float64

In [37]:
similarity

user_id
5      0.355491
13     0.387957
15     0.164938
38     0.259076
43     0.411448
         ...   
896    0.436883
907    0.251778
932    0.343172
934    0.365485
937    0.104322
Name: 882, Length: 85, dtype: float64

In [41]:
weighted_average_result_df

,user_id,movie_id,rating,predictedRating
0,868,358,2,3.701265
1,773,127,5,3.701265
2,332,566,4,3.701265
3,654,124,4,3.701265
4,775,305,4,3.701265
...,...,...,...,...
12997,600,181,4,3.701265
12998,65,73,4,3.701265
12999,686,23,5,3.701265
13000,715,462,4,3.701265


In [43]:
weighted_average_result_df.rating.unique()

array([2, 5, 4, 3, 1])

In [44]:
weighted_average_result_df.predictedRating.unique()

array([3.70126524])

####평가

In [42]:
# 성능구하기
from sklearn.metrics import mean_squared_error, mean_absolute_error

# rmse
mse = mean_squared_error(y_true = weighted_average_result_df['rating'].values,
                         y_pred = weighted_average_result_df['predictedRating'].values)
rmse = np.sqrt(mse)

# mae
mae = mean_absolute_error(y_true = weighted_average_result_df['rating'].values,
                          y_pred = weighted_average_result_df['predictedRating'].values)

print('rmse', rmse)
print('mae', mae)

rmse 1.0607580083181063
mae 0.8693956946018255


#### 추천

In [ ]:
# 전체 데이터 대상 적용?


In [ ]:
# recommender(user_id, num of recommendation)
def recommender_cf(user_id, num):
  weighted_average_result_df[weighted_average_result_df['user_id']== user_id].sort_values(['predictedRating'])